### This script loads subjects, gets evoked EEG traces for all events, adds a speed column to the stim log, and saves the files in each subject's data folder.

In [1]:
import os
import sys
import json
import time
import gspread
import pickle
import numpy as np
import pandas as pd
from scipy import signal, fftpack, stats, ndimage
import statsmodels.stats.multitest as multitest

In [2]:
sys.path.append(r'C:\Users\lesliec\code')

In [3]:
from tbd_eeg.tbd_eeg.data_analysis.eegutils import EEGexp
from tbd_eeg.tbd_eeg.data_analysis.Utilities.utilities import (
    get_stim_events,
    get_evoked_traces,
    get_evoked_firing_rates,
    find_nearest_ind
)
from allensdk.brain_observatory.ecephys.lfp_subsampling.subsampling import remove_lfp_offset
from allensdk.core.mouse_connectivity_cache import MouseConnectivityCache

In [4]:
%matplotlib notebook

Load CCF for identifying cortical areas

In [5]:
mcc = MouseConnectivityCache(resolution=10)
str_tree = mcc.get_structure_tree()

Load Zap_Zip-log_exp to get metadata for experiments

In [6]:
_gc = gspread.service_account() # need a key file to access the account
_sh = _gc.open('Zap_Zip-log_exp') # open the spreadsheet
_df = pd.DataFrame(_sh.sheet1.get()) # load the first worksheet
zzmetadata = _df.T.set_index(0).T # put it in a nicely formatted dataframe

#### Functions

### Load subjects from file

In [7]:
with open(r'C:\Users\lesliec\OneDrive - Allen Institute\data\all_EEG_subjects.json') as subjects_file:
    multi_sub_dict = json.load(subjects_file)

### Load EEG_exp and gather EEG traces

In [8]:
overwrite_existing_files = False

before_event = 2. # s, look at 2 s pre-stim
after_event = 2. # s, look at 2 s post-stim

apply_mask = True
apply_hpass = True
apply_lpass = True

In [9]:
for mouse_num, rec_folder in multi_sub_dict.items():
    print('{}'.format(mouse_num))
    exp = EEGexp(rec_folder, preprocess=False, make_stim_csv=False)
    exp_tag = exp.experiment_folder[exp.experiment_folder.find('mouse')+12:exp.experiment_folder.find(str(exp.date.year))-1]
    print(' {}'.format(exp_tag))
    
    ## Set file names ##
    evoked_folder = os.path.join(exp.data_folder, 'evoked_data')
    if not os.path.exists(evoked_folder):
        os.mkdir(evoked_folder)
    event_running_file = os.path.join(evoked_folder, 'event_running_speed.npy')
    event_running_ts_file = os.path.join(evoked_folder, 'event_running_times.npy')
    event_EEGtraces_file = os.path.join(evoked_folder, 'event_EEGtraces.npy')
    event_EEGtraces_ts_file = os.path.join(evoked_folder, 'event_EEGtraces_times.npy')
    
    ## Load stim log ##
    stim_log = pd.read_csv(exp.stimulus_log_file)
    stim_log = stim_log.astype({'parameter': str})
    all_event_times = stim_log['onset'].values
    
    ## Load running signal and get mean event speed ##
    if not os.path.exists(event_running_file) or overwrite_existing_files:
        print('  Getting event-related running...')
        run_signal, run_timestamps = exp.load_running()
        rinds = np.arange(-int(before_event * 100), int(after_event * 100))
        event_inds = np.array([find_nearest_ind(run_timestamps, x) for x in all_event_times])
        event_run_speed = run_signal[np.repeat([rinds], len(event_inds), axis=0).T + event_inds]
        event_run_times = rinds / 100
        ## Save ##
        np.save(event_running_file, event_run_speed, allow_pickle=False)
        np.save(event_running_ts_file, event_run_times, allow_pickle=False)
        ## Add speed to stim_log ##
        evinds = np.nonzero((event_run_times >= -0.5) & (event_run_times < 0.5))[0]
        mean_speed = np.mean(event_run_speed[evinds, :], axis=0)
        stim_log['mean_speed'] = mean_speed
        stim_log['resting_trial'] = stim_log['mean_speed'] == 0
        stim_log.to_csv(exp.stimulus_log_file, index=False)
        
    if not os.path.exists(event_EEGtraces_file) or overwrite_existing_files:
        ## Load EEG data and preprocess ##
        print('  Loading EEG data...')
        datai, tsi = exp.load_eegdata()
        eeg_chs = np.arange(0, datai.shape[1])

        ## Mask estim artifact ##
        if apply_mask:
            mask_samples = int(0.002 * exp.ephys_params['EEG']['sample_rate'])
            for etime in stim_log.loc[stim_log['stim_type'] == 'biphasic', 'onset'].to_numpy():
                val = find_nearest_ind(tsi, etime) - 2
                datai[val:val+mask_samples, :] = datai[val:val-mask_samples:-1, :]

        ## Apply high-pass filter ##
        if apply_hpass:
            hpb, hpa = signal.butter(3, 0.1/(exp.ephys_params['EEG']['sample_rate']/2), btype='highpass')
            datai = signal.filtfilt(hpb, hpa, datai, axis=0)

        ## Get evoked traces ##
        print('  Getting EEG traces...')
        event_traces, event_ts = get_evoked_traces(
            datai, tsi, all_event_times, before_event, after_event, exp.ephys_params['EEG']['sample_rate'])

        ## Apply lowpass filter ##
        if apply_lpass:
            lpb, lpa = signal.butter(3, 100/(exp.ephys_params['EEG']['sample_rate']/2), btype='low')
            event_traces = signal.filtfilt(lpb, lpa, event_traces, axis=0)

        ## Save ##
        print('   ...saving {}.'.format(event_EEGtraces_file))
        np.save(event_EEGtraces_file, event_traces, allow_pickle=False)
        np.save(event_EEGtraces_ts_file, event_ts, allow_pickle=False)
    else:
        print('  Not creating EEG traces file, it already exists.')
    
    print('')

521885
Experiment type: electrical stimulation
SomnoSuite log file not found.
 estim1
  Not creating EEG traces file, it already exists.

521886
Experiment type: electrical stimulation
SomnoSuite log file not found.
 estim1
  Not creating EEG traces file, it already exists.

521887
Experiment type: electrical stimulation
SomnoSuite log file not found.
 estim1
  Not creating EEG traces file, it already exists.

543393
Experiment type: electrical stimulation
SomnoSuite log file not found.
 estim1
  Not creating EEG traces file, it already exists.

543394
Experiment type: electrical stimulation
SomnoSuite log file not found.
 estim1
  Not creating EEG traces file, it already exists.

543395
Experiment type: electrical stimulation
SomnoSuite log file not found.
 estim1
  Not creating EEG traces file, it already exists.

543396
Experiment type: electrical and sensory stimulation
SomnoSuite log file not found.
 estim_vis1
  Not creating EEG traces file, it already exists.

546655
Experiment 